In [1]:
import torch
import gpytorch
from botorch.models import SingleTaskGP
from botorch.acquisition.analytic import ExpectedImprovement
from botorch.models.transforms.outcome import Standardize
from gpytorch.kernels import ScaleKernel
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_model
from botorch.optim.fit import fit_gpytorch_torch # fix Cholecky jitter error
from scipy.stats import norm
from sklearn.decomposition import PCA
import math 
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pickle
import os
import time

# config plot settings
plt.rcParams["font.size"] = 16

In [2]:
###
#  Load Data
###
file = h5py.File("targets_and_normalized_features.jld2", "r")
# feature matrix
X = torch.from_numpy(np.transpose(file["X"][:]))
# simulation data (high fidelity)
y = torch.from_numpy(np.transpose(file["gcmc_y"][:]))
# associated simulation costs
cost = torch.from_numpy(np.transpose(file["gcmc_elapsed_time"][:]))

# total number of COFs in data set
nb_COFs = X.shape[0]

print("raw data - \n\tX:", X.shape)
print("\t\ty:", y.shape)
print("\t\tcost: ", cost.shape)    
    
print("\nEnsure features are normalized - ")
print("max:\n", torch.max(X, 0).values)
print("min:\n", torch.min(X, 0).values)

raw data - 
	X: torch.Size([608, 14])
		y: torch.Size([608])
		cost:  torch.Size([608])

Ensure features are normalized - 
max:
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=torch.float64)
min:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64)


In [3]:
X_unsqueezed = X.unsqueeze(1)

## Helper Functions

#### Construct Initial Inputs

In [4]:
# find COF closest to the center of feature space
def get_initializing_COF(X):
    # center of feature space
    feature_center = np.ones(X.shape[1]) * 0.5
    # max possible distance between normalized features
    return np.argmin(np.linalg.norm(X - feature_center, axis=1))

# yields np.array([25, 494, 523])
def diverse_set(X, train_size):
    # initialize with one random point; pick others in a max diverse fashion
    ids_train = [get_initializing_COF(X)]
    # select remaining training points
    for j in range(train_size - 1):
        # for each point in data set, compute its min dist to training set
        dist_to_train_set = np.linalg.norm(X - X[ids_train, None, :], axis=2)
        assert np.shape(dist_to_train_set) == (len(ids_train), nb_COFs)
        min_dist_to_a_training_pt = np.min(dist_to_train_set, axis=0)
        assert np.size(min_dist_to_a_training_pt) == nb_COFs
        
        # acquire point with max(min distance to train set) i.e. Furthest from train set
        ids_train.append(np.argmax(min_dist_to_a_training_pt))
    assert np.size(np.unique(ids_train)) == train_size # must be unique
    return np.array(ids_train)

In [5]:
# construct feature matrix of acquired points
def build_X_train(ids_acquired):
    return X[ids_acquired, :]

# construct output vector for acquired points
def build_y_train(ids_acquired):
    return y[ids_acquired].unsqueeze(-1)

# construct vector to track accumulated cost of acquired points
def build_cost(ids_acquired):
    return cost[ids_acquired]

In [6]:
# construct and fit GP model
def construct_and_fit_gp_model(X_train, y_train):      
    model = SingleTaskGP(X_train, y_train, outcome_transform=Standardize(m=1))
    mll   = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_model(mll, optimizer=fit_gpytorch_torch)
    return model

#### Bayesian Algorithm

In [7]:
def run_Bayesian_optimization(nb_iterations, nb_COFs_initialization):
    assert nb_iterations > nb_COFs_initialization
    ###
    #  Initialize system
    ###
    # select initial COFs for training data
    ids_acquired = diverse_set(X, nb_COFs_initialization)
    
    ###
    #  itterate through remaining budget using BO
    ###
    for i in range(nb_COFs_initialization, nb_iterations):
        # construct training data
        X_train = build_X_train(ids_acquired)
        y_train = build_y_train(ids_acquired)
        
        # fit GP surrogate model
        model = construct_and_fit_gp_model(X_train, y_train)
        
        # compute expected improvement acquisition function
        acq_fn   = ExpectedImprovement(model, best_f=y_train.max().item())
        acq_vals = acq_fn.forward(X.unsqueeze(1))
        
        # acquire next COF with highest acquisition value currently not in training set
        ids_sorted_by_aquisition = acq_vals.argsort(descending=True)
        for id_max in ids_sorted_by_aquisition:
            if not id_max.item() in ids_acquired:
                id_max_acq = id_max.item()
                break
                
        ###
        #  acquire this COF
        ###
        ids_acquired = np.concatenate((ids_acquired, [id_max_acq]))
        print("BO iteration ", i)
        print("\tacquired COF ", id_max_acq)
        print("\ty = ", y[id_max_acq].item())
    
    # check budget constraint is stisfied
    assert np.size(ids_acquired) == nb_iterations
    assert len(np.unique(ids_acquired)) == nb_iterations
    return ids_acquired

# Run BO

In [8]:
nb_iterations = 100
nb_COFs_initialization = 3

ids_acquired = run_Bayesian_optimization(nb_iterations, nb_COFs_initialization)

y_acquired    = y[ids_acquired]
cost_acquired = build_cost(ids_acquired)

Iter 10/100: 2.531564513532533
Iter 20/100: 1.5977845202378844
Iter 30/100: 1.533658962532855
Iter 40/100: 1.5103386884289904
Iter 50/100: 1.4671705627878475
Iter 60/100: 1.464220585531777
BO iteration  3
	acquired COF  521
	y =  15.766064256252303
Iter 10/100: 2.259921953527158
Iter 20/100: 1.536637681235032


/home/cokes/anaconda3/lib/python3.9/site-packages/botorch/fit.py:148: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2189.)
  warnings.warn(w.message, w.category)


Iter 30/100: 1.4498535831166488
Iter 40/100: 1.42968022001905
BO iteration  4
	acquired COF  522
	y =  12.350590208958222
Iter 10/100: 2.109082060927381
Iter 20/100: 1.5259160914647434
Iter 30/100: 1.4413372951204617
Iter 40/100: 1.4416352305317903
BO iteration  5
	acquired COF  524
	y =  6.895659242359536
Iter 10/100: 2.0135324701675734
Iter 20/100: 1.5207324687563244
Iter 30/100: 1.432088136389039
Iter 40/100: 1.4002056503388152
BO iteration  6
	acquired COF  583
	y =  2.903458451797101
Iter 10/100: 1.947059639754468
Iter 20/100: 1.5146320272437355
Iter 30/100: 1.4179314352311199
Iter 40/100: 1.3044569395438106
Iter 50/100: 1.2702834261229834
Iter 60/100: 1.2677354117770834
BO iteration  7
	acquired COF  223
	y =  4.3554296195824325
Iter 10/100: 1.8931670383400077
Iter 20/100: 1.5054804000377129
Iter 30/100: 1.4023125420516471
Iter 40/100: 1.2787566379596629
BO iteration  8
	acquired COF  0
	y =  1.696244892692333
Iter 10/100: 1.8515727575576189
Iter 20/100: 1.495702794354195
Iter 30

In [9]:
bo_iter_top_cof_acquired = np.argmax(y_acquired == max(y))
top_cof_acc_cost = sum(cost_acquired[:bo_iter_top_cof_acquired]).item() # 10978.643530249596 w/o optimizer

print("iteration we acquire top COF = ", bo_iter_top_cof_acquired.item() + 1)
print("accumulated cost up to observation of top COF = ", top_cof_acc_cost, " [min]" )

iteration we acquire top COF =  22
accumulated cost up to observation of top COF =  6759.435011923313  [min]


In [10]:
###
#  Store SFBO results
###
sfbo_res = dict({'ids_acquired': ids_acquired,
                'y_acquired': y_acquired,
                'cost_acquired': cost_acquired
                })

with open('search_results/sfbo_results_with_EI.pkl', 'wb') as file:
    pickle.dump(sfbo_res, file)

# Random Search

In [11]:
nb_runs = 1000

rs_res = dict()
rs_res['ids_acquired'] = []
rs_res['cost_acquired'] = []

for r in range(nb_runs):
    rs_ids_acquired = np.random.choice(range(nb_COFs), replace=False, size=nb_iterations)
    rs_cost_acquired = build_cost(rs_ids_acquired)
    rs_res['ids_acquired'].append(rs_ids_acquired)
    rs_res['cost_acquired'].append(rs_cost_acquired)

In [12]:
def test_rs(rs_res):
    for a in np.random.choice(nb_runs, replace=False, size=7):
        # check that the lengths 
        assert len(rs_res['ids_acquired'][a]) == nb_iterations
        assert all([cof_id in range(nb_COFs) for cof_id in rs_res['ids_acquired'][a]])
    return

test_rs(rs_res)

In [18]:
rs_res['ids_acquired'][1][:4]

array([ 43, 472, 405, 550])

In [ ]:
# get y_max acquired up to iteration i for i = 1,2,...
def y_max(rs_ids_acquired):
    y_max_mu      = np.zeros(nb_iterations)
    y_max_sig_bot = np.zeros(nb_iterations)
    y_max_sig_top = np.zeros(nb_iterations)
    # element i of these will be y max at BO iter i
    
    for i in range(1, nb_iterations+1):
        # max value acquired up to this point
        y_maxes = np.array([max(y[rs_ids_acquired[r][:i]]) for r in range(nb_runs)])
        assert np.size(y_maxes) == nb_runs
        y_max_mu[i-1]      = np.mean(y_maxes)
        y_max_sig_bot[i-1] = np.std(y_maxes[y_maxes < y_max_mu[i-1]])
        y_max_sig_top[i-1] = np.std(y_maxes[y_maxes > y_max_mu[i-1]])
    return y_max_mu, y_max_sig_bot, y_max_sig_top

# rs_mean, rs_lower_bound, rs_upper_bound = y_max(rs_res)
y_rs_max_mu, y_rs_max_sig_bot, y_rs_max_sig_top = y_max(rs_res['ids_acquired'])

In [14]:
###
#  Store Random Search Results
###
random_search_results = dict({'ids_acquired': rs_res['ids_acquired'],
                             'y_rs_max_mu': y_rs_max_mu,
                             'y_rs_max_sig_bot': y_rs_max_sig_bot,
                             'y_rs_max_sig_top': y_rs_max_sig_top,
                             'cost_acquired': rs_res['cost_acquired']
                             })

with open('search_results/random_search_results.pkl', 'wb') as file:
    pickle.dump(random_search_results, file)